**xlnet Training**

Importing useful libraries at first

In [1]:
import torch
torch.cuda.empty_cache()

In [2]:
import pandas as pd
import numpy as np

import sklearn
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns
%matplotlib inline
pd.options.display.max_colwidth = 200
pd.options.display.max_columns = 200
SEED = 100

import warnings
warnings.filterwarnings("ignore")

import pickle
import time

import re
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import ToktokTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from string import punctuation

import torch
from torch import nn ,cuda
from torch.utils.data import DataLoader,Dataset,RandomSampler, SequentialSampler

np.random.seed(SEED)
num_labels =  15
pos_weight = 10

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1" 

In [5]:
# to check if PyTorch is capable of using GPU, run the following code.

import torch
torch.cuda.is_available()
# Output would be True if Pytorch is using GPU otherwise it would be False.

True

In [6]:
# Importing the database 

with open('/content/drive/MyDrive/thesis/feature_eng_dataframe.pkl', 'rb') as f:
    df_train,df_val,df_test,tag_cols = pickle.load(f)

In [7]:
df_train.head(5)

,text,labels
6867,place need compare 2 nullable value see think something framework support find anything instead follow public static bool isdifferentto bool x bool return xhasvalue yhasvalue true xhasvalue amp xv...,"(0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)"
18494,codehtml file contain follow code ajax type post datatype jsonp url path success function msg var e documentcreateelement div eid ads documentbodyappendchild e ads html msg open codehtml file brow...,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0)"
29363,check whether bootstrapjs load page file bootstrapjs may compile another big js file,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0)"
36595,try run wordcount job hadoopbut always get class find exceptioni post class write command use run job import javaioioexception import javautil import orgapachehadoopfspath import orgapachehadoopco...,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0)"
24197,logcat see lot dsr send dtr android tether phone wirelessly mac internet dev mode also mean see something bluetooth though dsr dtr,"(0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)"


### Modelling - SimpleTransformers


In [8]:
!pip install simpletransformers
!pip install h5py
!pip install wandb --upgrade  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 KB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 83.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 96.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 91.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 96.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [9]:
from simpletransformers.classification import MultiLabelClassificationModel
from simpletransformers.classification import classification_model
import logging
import wandb

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

Defining ModelArgs for xlnet model

In [10]:
# The ModelArgs (train_args) dataclass contains all the global options set to their default values

sweep_config = {
    "name": "stacksample-hyperparameter-xlnet",
    "method": "bayes",  
    "metric": {"name": "eval_loss", "goal": "minimize"},
    "parameters": {
        "learning_rate": {"min": 3e-5, "max": 7e-5},
        "train_batch_size": {"values": [32, 64, 128]},
        "gradient_accumulation_steps": {"values": [8, 16]},
        "max_seq_length": {"values": [128, 256]},
    }
}

model_type = 'xlnet'
model_name = 'xlnet-base-cased'

sweep_id = wandb.sweep(sweep_config, project="stacksample-hyperparameter-xlnet")

train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,

    # paths
    "output_dir": f"/content/drive/MyDrive/thesis/output/xlnet_tuning/final",
    "best_model_dir": f"/content/drive/MyDrive/thesis/output/xlnet_tuning/best_model",

    # size
    #"train_batch_size": 64, # 
    #"max_seq_length": 128, # use small value to avoid OOM
    "num_train_epochs": 10,

    # rates
    # "weight_decay": 0,
    #"learning_rate": 3e-5,
    # "adam_epsilon": 1e-8,

    # evaluation
    "evaluate_during_training": True,
    "evaluate_during_training_steps": 1000,
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "evaluate_during_training_verbose": False,
    "sliding_window": False,
    #"eval_batch_size": 64,
    #"gradient_accumulation_steps": 16,
    "wandb_project" : "stacksample-hyperparameter-xlnet",
    "use_early_stopping":True,
    "early_stopping_patience": 5,
    "early_stopping_delta": 0,
    "early_stopping_metric": "eval_loss",
    "early_stopping_metric_minimize": True,

}

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: oxnt0d28
Sweep URL: https://wandb.ai/ranvir_rana/stacksample-hyperparameter-xlnet/sweeps/oxnt0d28


Initializing a ClassificationModel for xlnet model

In [11]:
def train():
    # Initialize a new wandb run
    wandb.init()

    # Create a TransformerModel
    model = MultiLabelClassificationModel(model_type, model_name,args=train_args,num_labels=num_labels, use_cuda=True,sweep_config=wandb.config)

    # Train the model
    model.train_model(df_train, eval_df=df_val)

    # Evaluate the model
    model.eval_model(df_val)

    # Sync wandb
    wandb.join()

In [ ]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: 82ci8q5u with config:
wandb: 	gradient_accumulation_steps: 16
wandb: 	learning_rate: 4.096141165104417e-05
wandb: 	max_seq_length: 128
wandb: 	train_batch_size: 64
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ranvir_rana. Use `wandb login --relogin` to force relogin


Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForMultiLabelSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForMultiLabelSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream tas

  0%|          | 0/26700 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/418 [00:00<?, ?it/s]

...

Run sgg8881y errored: OutOfMemoryError('CUDA out of memory. Tried to allocate 94.00 MiB (GPU 0; 14.75 GiB total capacity; 13.70 GiB already allocated; 12.81 MiB free; 13.73 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF')
wandb: ERROR Run sgg8881y errored: OutOfMemoryError('CUDA out of memory. Tried to allocate 94.00 MiB (GPU 0; 14.75 GiB total capacity; 13.70 GiB already allocated; 12.81 MiB free; 13.73 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF')
wandb: Agent Starting Run: kx09oq2g with config:
wandb: 	gradient_accumulation_steps: 8
wandb: 	learning_rate: 5.494536703781168e-05
wandb: 	max_seq_length: 256
wandb: 	train_batch_size: 128


Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForMultiLabelSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForMultiLabelSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream tas

Run kx09oq2g errored: OutOfMemoryError('CUDA out of memory. Tried to allocate 94.00 MiB (GPU 0; 14.75 GiB total capacity; 13.70 GiB already allocated; 12.81 MiB free; 13.73 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF')
wandb: ERROR Run kx09oq2g errored: OutOfMemoryError('CUDA out of memory. Tried to allocate 94.00 MiB (GPU 0; 14.75 GiB total capacity; 13.70 GiB already allocated; 12.81 MiB free; 13.73 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF')
wandb: Agent Starting Run: ou6kgq14 with config:
wandb: 	gradient_accumulation_steps: 8
wandb: 	learning_rate: 3.317386556438509e-05
wandb: 	max_seq_length: 128
wandb: 	train_batch_size: 64


Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForMultiLabelSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForMultiLabelSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream tas

Run ou6kgq14 errored: OutOfMemoryError('CUDA out of memory. Tried to allocate 94.00 MiB (GPU 0; 14.75 GiB total capacity; 13.70 GiB already allocated; 12.81 MiB free; 13.73 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF')
wandb: ERROR Run ou6kgq14 errored: OutOfMemoryError('CUDA out of memory. Tried to allocate 94.00 MiB (GPU 0; 14.75 GiB total capacity; 13.70 GiB already allocated; 12.81 MiB free; 13.73 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF')
wandb: Agent Starting Run: oshx98ao with config:
wandb: 	gradient_accumulation_steps: 8
wandb: 	learning_rate: 3.349528695730745e-05
wandb: 	max_seq_length: 128
wandb: 	train_batch_size: 128
